<a href="https://colab.research.google.com/github/babuyashwanth/deeplearning_ann/blob/main/HiddenLayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset=pd.read_csv('/content/Churn_Modelling.csv')

In [48]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([("Geography", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)

In [5]:
labelencoder_x_2=LabelEncoder()
X[:,4]=labelencoder_x_2.fit_transform(X[:,4])

In [6]:
X=X[:,1:]

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [8]:
X_train.shape

(8000, 11)

In [9]:
y_train.shape

(8000,)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test =sc.transform(X_test)

In [11]:
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization
from keras.activations import relu,sigmoid

In [19]:
def create_model(layers,activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(activation))
    
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
    model.add(Dense(1))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [20]:
model = KerasClassifier(build_fn=create_model,verbose=0)

In [21]:
model

In [22]:
layers=[[20],[15,30],[25,10,12]]
activations = ['sigmoid','relu']
param_grid = dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
grid = GridSearchCV(estimator=model,param_grid=param_grid,cv=5)

In [23]:
grid_result= grid.fit(X_train,y_train)

In [24]:
(grid_result.best_score_,grid_result.best_params_)

(0.8304999947547913,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': [25, 10, 12]})

In [25]:
y_pred=grid.predict(X_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [27]:
y1_pred=y_pred>0.5

In [28]:
y1_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [29]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y1_pred)

In [30]:
cm

array([[1531,   64],
       [ 211,  194]])

In [31]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y1_pred)

In [32]:
score

0.8625